# **ETL del archivo Lesiones_Hechos**

## **1.Cargando Las Librerías Necesarias**

In [ ]:
import pandas as pd #Cargaremos pandas para manejar los datos de nuestros datasets,
import numpy as np #Cargaremos numpy para manejar los datos,
import gzip #Cargaremos gzip para poder abrir archivos con compresion gzip",
import datetime #Cargaremos datetime para manejar fechas",
from google.colab import drive #Cargaremos drive para poder acceder a los archivos de drive"
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
url01="/content/drive/MyDrive/2023/Henry/PI02-DataAnalyst/Datos/lesiones.xlsx"
url02="/content/drive/MyDrive/2023/Henry/PI02-DataAnalyst/Datos/lesiones_hechos.csv"

## **2.Fase ETL**

### 2.1.Carga de Datos

In [ ]:
df_lesiones_hechos=pd.read_excel(url01,sheet_name="HECHOS")

In [ ]:
df_lesiones_hechos.to_csv(url02,index=True,header=True,encoding='utf-8')

In [ ]:
df_lesiones_hechos=pd.read_csv(url02,encoding='utf-8')

In [ ]:
df_lesiones_hechos.head()

,Unnamed: 0,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9,SD,...,-34.559658,CICLISTA,SD,CICLISTA-SD,SD,SD,SD,SD,x,SD
1,1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1,SD,...,-34.669125,AUTO,SD,AUTO-SD,SD,x,SD,SD,SD,SD
2,2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2,SD,...,-34.677556,SD,SD,SD-SD,SD,SD,SD,SD,SD,SD
3,3,LC-2019-0000079,1,2019,1,1,2019-01-01 00:00:00,02:30:00,2,SD,...,-34.647349,PEATON,SD,PEATON-SD,x,SD,SD,SD,SD,SD
4,4,LC-2019-0000082,4,2019,1,1,2019-01-01 00:00:00,04:30:00,4,SD,...,-34.604579,AUTO,SD,AUTO-SD,SD,SD,x,SD,SD,SD


### 2.2.Analisis Descriptivo De Los Datos

#### 2.2.1.Funciones para el analisis descriptivo de los datos

In [ ]:
def check_df(dataset, head = 5):

  """
  Funcion creada para tener una vista general de la base de datos
  """

  print('*'*30 + 'Forma de la base de datos' + '*'*30, end = '\n'*2)
  print(dataset.shape, end = '\n'*2)#Tamaño de la base de datos

  print('*'*30 + 'Informacion general de la base de datos' + '*'*30, end = '\n'*2)
  print(dataset.info(), end = '\n'*2)# Informacion general

  print('*'*30 + 'NaN cantidad de nulos' + '*'*30, end = '\n'*2)
  print(dataset.isnull().sum(), end = '\n'*2)#Cuenta la cantidad de NaN por columna

#### 2.2.2.Descripcion De Los Datos

In [ ]:
check_df(df_lesiones_hechos)

******************************Forma de la base de datos******************************

(23785, 28)

******************************Informacion general de la base de datos******************************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             23785 non-null  int64  
 1   id                     23785 non-null  object 
 2   n_victimas             23785 non-null  int64  
 3   aaaa                   23785 non-null  int64  
 4   mm                     23785 non-null  int64  
 5   dd                     23785 non-null  int64  
 6   fecha                  23785 non-null  object 
 7   hora                   23785 non-null  object 
 8   franja_hora            23780 non-null  object 
 9   direccion_normalizada  23732 non-null  object 
 10  comuna                 23616 non-null  object 
 11  tipo_call

### 2.3.Transformacion De Los Datos

#### 2.3.1.Funciones para la transformacion de los datos

In [ ]:
def Transform_Hora(text):

  '''

  La funcion de Transform_Hora recibe como parametro un dato de cualquier tipo
  el cual pasara a trasnformarse en texto y devuelve un objeto de tipo datetime
  en formato de 24horas.
  Observando 3 formatos de datos en total 2 de formato fecha y un tipo de dato
  string entonces procedemos de la siguiente manera :

  format_12h= "%I:%M:%S %p"
  format_24h='%H:%M:%S'
  format_sd='sd'
  Para este ejercicico lo pasaremos a formato 24 horas que es el formato
   por defecto

  '''
  text=str(text)
  format_12h= "%I:%M:%S %p"
  format_24h='%H:%M:%S'
  if (text.count("p.m.") or text.count("a.m."))>0: #format 12h
    text=text.replace("p.m.","PM").replace("a.m.","AM")
    time12h = datetime.datetime.strptime(text,format_12h).time()
    time24h = time12h.strftime(format_24h)
    time24h=datetime.datetime.strptime(time24h,format_24h).time()
    return  time24h
  elif text.count(":")==2: #format 24h
    time24h = datetime.datetime.strptime(text, format_24h).time()
    return  time24h

  else:
    return float(np.nan)


def reemplazar_nan(value):
  '''

  La funcion reemplazar_nan busca reemplaza las palabras como 'SD' , 'sd o
  No Especificada' por
  valores NaN

  '''
  if value in ('SD', 'sd', 'No Especificada'):
      return float(np.nan)
  else:
      return value

def reemplazar_participacion(value):
  '''

  La funcion reemplazar_participacion busca reemplaza en las columnas designadas
  para los participanes del siniestro como ['MOTO','AUTO','TRANSPORTE_PUBLICO',
  'CAMION','CICLISTA']
  valores NaN

  '''
  if value=='x':
    return round(1,0)
  elif value=='0':
    return round(0,0)
  else:
      return value


def Transform_Dir(text):
  """

  Esta funcion esta diseñada para reordenar la columna LUGAR_DEL_HECHO teniendo
  como principla estructura teniendo como prefijo AV. y DR. y eliminando la coma
  normaliando las abreviaciones en general

  """
  if isinstance(text, str):
    text=str(text).upper()
    if "," in text:
      parts = [part.strip() for part in text.split(",")]
      if len(parts) == 2:
          text= f"{parts[1]} {parts[0]} "
      if len(parts) == 3:
          text= f"{parts[2]} {parts[1]} {parts[0]} "
    if "DR." in text:
      parts = [part.strip() for part in text.split("DR.")]

      if len(parts) == 2:
        text= f"DR. {parts[0]} {parts[1]}"

    if "AV." in text:
      parts = [part.strip() for part in text.split("AV.")]

      if len(parts) == 2:
        text= f"AV. {parts[0]} {parts[1]}"

    text=text.replace('GRAL.','GENERAL').replace('TTE.','TENIENTE').replace('CNEL.','CORONEL')
    text=text.strip()

    return text
  else:
    return float(np.nan)

#### 2.3.2.Transformacion de los datos

Se reviso las primeras las primeras 5  columnas **'ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD'** y no muestran errores como tal

In [ ]:
df_lesiones_hechos.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
df_lesiones_hechos.columns=['ID', 'N_VICTIMAS', 'AAAA', 'MM', 'DD','FECHA', 'HORA', 'HH','DIRECCION NORMALIZADA','COMUNA','TIPO_DE_CALLE','LUGAR_DEL_HECHO','CALLE', 'ALTURA', 'CRUCE',
                             'XY (CABA)', 'LONGITUD', 'LATITUD','VICTIMA','ACUSADO','PARTICIPANTES','MOTO','AUTO','TRANSPORTE_PUBLICO','CAMION','CICLISTA','GRAVEDAD']


In [ ]:
df_lesiones_hechos.head()

,ID,N_VICTIMAS,AAAA,MM,DD,FECHA,HORA,HH,DIRECCION NORMALIZADA,COMUNA,...,LATITUD,VICTIMA,ACUSADO,PARTICIPANTES,MOTO,AUTO,TRANSPORTE_PUBLICO,CAMION,CICLISTA,GRAVEDAD
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9,SD,14,...,-34.559658,CICLISTA,SD,CICLISTA-SD,SD,SD,SD,SD,x,SD
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1,SD,8,...,-34.669125,AUTO,SD,AUTO-SD,SD,x,SD,SD,SD,SD
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2,SD,8,...,-34.677556,SD,SD,SD-SD,SD,SD,SD,SD,SD,SD
3,LC-2019-0000079,1,2019,1,1,2019-01-01 00:00:00,02:30:00,2,SD,7,...,-34.647349,PEATON,SD,PEATON-SD,x,SD,SD,SD,SD,SD
4,LC-2019-0000082,4,2019,1,1,2019-01-01 00:00:00,04:30:00,4,SD,3,...,-34.604579,AUTO,SD,AUTO-SD,SD,SD,x,SD,SD,SD


Observamos que existe una gran cantidad de SD definidos como no definir sin embargo para nuestro analisis lo consideraremos como NaN y posteriormente haremos un **dropna** para poder procesar los datos

In [ ]:
#Seleccionamos las columnas que nos interesan por su gran volumen de 'SD'
columnas_modificar=['HORA','HH','COMUNA','LATITUD','LONGITUD','VICTIMA','ACUSADO','MOTO','CAMION','AUTO','TRANSPORTE_PUBLICO','CICLISTA','GRAVEDAD','DIRECCION NORMALIZADA','CRUCE','CALLE']

In [ ]:
#Reemplazaremos las etiquetas de SD con NaN
df_lesiones_hechos=df_lesiones_hechos.applymap(reemplazar_nan)

In [ ]:
#Tenemos algunas fechas en un formato diferente por tanto procederemos a normalizarlas
df_lesiones_hechos['FECHA'] = df_lesiones_hechos['FECHA'].apply(
    lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date() if x.count("-") >= 2 else (datetime.datetime.strptime(x, '%m/%d/%Y').date() if x.count("/")>=2 else float(np.nan)))

In [ ]:
#Eliminaremos las filas que no tengan los valores de ubicacion
df_lesiones_hechos=df_lesiones_hechos.dropna(thresh=2,subset=['LATITUD','LONGITUD'])


In [ ]:
#reemplazamos las etiquetas de participacion
df_lesiones_hechos[['MOTO','CAMION','AUTO','TRANSPORTE_PUBLICO','CICLISTA']]=df_lesiones_hechos[['MOTO','CAMION','AUTO','TRANSPORTE_PUBLICO','CICLISTA']].applymap(reemplazar_participacion)

In [ ]:
#Transformamos las horas a formato 24 horas
df_lesiones_hechos['HORA']=df_lesiones_hechos['HORA'].apply(Transform_Hora)

In [ ]:
#Cambiamos el formato de la columna
df_lesiones_hechos['HH']=pd.to_numeric(df_lesiones_hechos['HH'])

In [ ]:
#Procederemos a normalizar las direcciones
df_lesiones_hechos['CALLE']=df_lesiones_hechos['CALLE'].apply(Transform_Dir)
df_lesiones_hechos['CRUCE']=df_lesiones_hechos['CRUCE'].apply(Transform_Dir)

In [ ]:
#Procederemos a reemplazar la direccion por nuestras columnas ya normalizadas
df_lesiones_hechos['DIRECCION NORMALIZADA'] = df_lesiones_hechos.apply(
    lambda columna: (columna["CALLE"] + " Y " + columna["CRUCE"]) if (isinstance(columna["CRUCE"], str) and isinstance(columna["CALLE"],str))
    else (columna["CALLE"] if isinstance(columna["CALLE"],str)
          else (columna["CRUCE"] if isinstance(columna["CRUCE"], str)
                else columna["LUGAR_DEL_HECHO"]))
    ,axis=1)


In [ ]:
#Colocamos en mayusculas todas las direcciones
df_lesiones_hechos['DIRECCION NORMALIZADA']=df_lesiones_hechos['DIRECCION NORMALIZADA'].apply(lambda x: x.upper() if isinstance(x,str) else x)

In [ ]:
#Cambiaremos algunas variaciones que nos muestra la columna GRAVEDAD como mayusculas y minusculas
df_lesiones_hechos['GRAVEDAD']=df_lesiones_hechos['GRAVEDAD'].apply(lambda x: x.upper() if isinstance(x,str) else x)

In [ ]:
df_lesiones_hechos

,ID,N_VICTIMAS,AAAA,MM,DD,FECHA,HORA,HH,DIRECCION NORMALIZADA,COMUNA,...,LATITUD,VICTIMA,ACUSADO,PARTICIPANTES,MOTO,AUTO,TRANSPORTE_PUBLICO,CAMION,CICLISTA,GRAVEDAD
0,LC-2019-0000179,1,2019,1,1,2019-01-01,09:00:00,9,"OBLIGADO RAFAEL, AV.COSTANERA 3300, BARRIO: PA...",14,...,-34.559658,CICLISTA,NaN,CICLISTA-SD,NaN,NaN,NaN,NaN,1.0,NaN
1,LC-2019-0000053,1,2019,1,1,2019-01-01,01:55:00,1,"AUTOPISTA PRESIDENTE HÉCTOR J. CÁMPORA Y ROCA,...",8,...,-34.669125,AUTO,NaN,AUTO-SD,NaN,1.0,NaN,NaN,NaN,NaN
2,LC-2019-0000063,1,2019,1,1,2019-01-01,02:00:00,2,"BARROS PAZOS, JOSE Y LARRAYA 0, BARRIO: VILLA ...",8,...,-34.677556,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
3,LC-2019-0000079,1,2019,1,1,2019-01-01,02:30:00,2,"TORRES Y TENORIO, PRES. Y RIESTRA AV. 0, BARRI...",7,...,-34.647349,PEATON,NaN,PEATON-SD,1.0,NaN,NaN,NaN,NaN,NaN
4,LC-2019-0000082,4,2019,1,1,2019-01-01,04:30:00,4,"CORRIENTES AV. Y URIBURU JOSE E., PRES. 0, BAR...",3,...,-34.604579,AUTO,NaN,AUTO-SD,NaN,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23780,LC-2021-0652849,1,2021,12,31,2021-12-31,19:30:00,19,AV. JUAN BAUTISTA ALBERDI Y GUAMINI,9,...,-34.659713573880,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
23781,LC-2021-0652865,2,2021,12,31,2021-12-31,19:40:00,19,AV. JUAN BAUTISTA ALBERDI,9,...,-34.641753304864,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
23782,LC-2021-0652907,1,2021,12,31,2021-12-31,20:00:00,20,BARRIO PADRE CARLOS MUGICA (VILLA 31 BIS),1,...,-34.583083,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
23783,LC-2021-0652921,1,2021,12,31,2021-12-31,22:00:00,22,LINIERS VIRREY Y MORENO,5,...,-34.614288229345,MOTO,TRANSPORTE PUBLICO,MOTO-TRANSPORTE PUBLICO,1.0,0.0,1.0,0.0,0.0,GRAVE


In [ ]:
df_lesiones_hechos.drop(columns=['XY (CABA)','LUGAR_DEL_HECHO','PARTICIPANTES'],inplace=True)

In [ ]:
df_lesiones_hechos

,ID,N_VICTIMAS,AAAA,MM,DD,FECHA,HORA,HH,DIRECCION NORMALIZADA,COMUNA,...,LONGITUD,LATITUD,VICTIMA,ACUSADO,MOTO,AUTO,TRANSPORTE_PUBLICO,CAMION,CICLISTA,GRAVEDAD
0,LC-2019-0000179,1,2019,1,1,2019-01-01,09:00:00,9,"OBLIGADO RAFAEL, AV.COSTANERA 3300, BARRIO: PA...",14,...,-58.408911,-34.559658,CICLISTA,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1,LC-2019-0000053,1,2019,1,1,2019-01-01,01:55:00,1,"AUTOPISTA PRESIDENTE HÉCTOR J. CÁMPORA Y ROCA,...",8,...,-58.44351,-34.669125,AUTO,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2,LC-2019-0000063,1,2019,1,1,2019-01-01,02:00:00,2,"BARROS PAZOS, JOSE Y LARRAYA 0, BARRIO: VILLA ...",8,...,-58.468335,-34.677556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LC-2019-0000079,1,2019,1,1,2019-01-01,02:30:00,2,"TORRES Y TENORIO, PRES. Y RIESTRA AV. 0, BARRI...",7,...,-58.437425,-34.647349,PEATON,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,LC-2019-0000082,4,2019,1,1,2019-01-01,04:30:00,4,"CORRIENTES AV. Y URIBURU JOSE E., PRES. 0, BAR...",3,...,-58.398225,-34.604579,AUTO,NaN,NaN,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23780,LC-2021-0652849,1,2021,12,31,2021-12-31,19:30:00,19,AV. JUAN BAUTISTA ALBERDI Y GUAMINI,9,...,-58.513477158887,-34.659713573880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23781,LC-2021-0652865,2,2021,12,31,2021-12-31,19:40:00,19,AV. JUAN BAUTISTA ALBERDI,9,...,-58.488326609395,-34.641753304864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23782,LC-2021-0652907,1,2021,12,31,2021-12-31,20:00:00,20,BARRIO PADRE CARLOS MUGICA (VILLA 31 BIS),1,...,-58.382894,-34.583083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23783,LC-2021-0652921,1,2021,12,31,2021-12-31,22:00:00,22,LINIERS VIRREY Y MORENO,5,...,-58.414532050899,-34.614288229345,MOTO,TRANSPORTE PUBLICO,1.0,0.0,1.0,0.0,0.0,GRAVE


## **3.Exportacion**

In [ ]:
path="/content/drive/MyDrive/2023/Henry/PI02-DataAnalyst/Data_Clean/lesiones_hechos.parquet.gzip"

In [ ]:
df_lesiones_hechos.to_parquet(path,compression="gzip")